In [ ]:
# 🛠️ Install dependencies
!pip install transformers datasets scikit-learn pandas

In [5]:
import pandas as pd
df = pd.read_csv('/content/synthetic_resume_dataset.csv')
df.head()

,text,label
0,Action everybody sell environment. Focus ahead...,Software Engineer
1,Exist gun hospital cover since seek over wall....,Software Engineer
2,Point street radio clear give grow safe natura...,Software Engineer
3,Both strong history behind here face above. Sk...,Software Engineer
4,Popular decision beyond help control represent...,Software Engineer


In [8]:
from preprocess import load_and_preprocess_data
train_ds, test_ds, label2id, id2label = load_and_preprocess_data("/content/synthetic_resume_dataset.csv", "bert-base-uncased")

Casting the dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [9]:
train_ds

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 80
})

In [13]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed,
)
import numpy as np

model_name = "bert-base-uncased"
output_dir = "/"

def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, f1_score
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

print("📦 Loading model...")
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    remove_unused_columns=False,  # ✅ IMPORTANT FIX
    logging_dir="../logs",
    logging_steps=10
)

print("🚀 Starting training...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📦 Loading model...
🚀 Starting training...


<ipython-input-13-817ca7367394>:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fishermorgan61 (fishermorgan61-mettersonbusiness) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.314100,2.340160,0.100000,0.018182
2,2.327800,2.256238,0.200000,0.123529
3,2.143200,2.059865,0.400000,0.304444
4,2.008500,1.904788,0.450000,0.350000
5,1.885300,1.821658,0.500000,0.465714
6,1.796800,1.705484,0.600000,0.527619
7,1.680100,1.610479,0.650000,0.613333
8,1.570600,1.541882,0.750000,0.726667
9,1.526100,1.490126,0.750000,0.726667
10,1.520900,1.480367,0.800000,0.773333


TrainOutput(global_step=100, training_loss=1.8773340797424316, metrics={'train_runtime': 1880.7012, 'train_samples_per_second': 0.425, 'train_steps_per_second': 0.053, 'total_flos': 52625990860800.0, 'train_loss': 1.8773340797424316, 'epoch': 10.0})

In [14]:
# Save
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")

# Download locally
!zip -r saved_model.zip saved_model
from google.colab import files
files.download("saved_model.zip")


  adding: saved_model/ (stored 0%)
  adding: saved_model/special_tokens_map.json (deflated 42%)
  adding: saved_model/vocab.txt (deflated 53%)
  adding: saved_model/model.safetensors (deflated 7%)
  adding: saved_model/tokenizer.json (deflated 71%)
  adding: saved_model/config.json (deflated 56%)
  adding: saved_model/tokenizer_config.json (deflated 75%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>